In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('./data/NG1_data_new.xlsx')
data['Dates'] = pd.to_datetime(data['Dates'])
data = data.sort_values('Dates').reset_index(drop=True)
data

,Dates,NG1 Comdty,NG2 Comdty,NG3 Comdty,NG4 Comdty,NG6 Comdty
0,2004-11-01,2183.719,4642.824,1554.613,209.742,59.307
1,2004-11-02,2145.403,4568.689,1532.367,206.695,58.850
2,2004-11-03,2191.732,4645.295,1558.733,210.211,59.481
3,2004-11-04,2053.246,4398.178,1479.635,201.774,58.725
4,2004-11-05,1991.892,4215.312,1413.719,193.102,57.313
...,...,...,...,...,...,...
5235,2024-11-25,3.404,3.288,3.008,2.952,3.184
5236,2024-11-26,3.467,3.311,3.040,2.996,3.228
5237,2024-11-27,3.204,3.069,2.843,2.827,3.088
5238,2024-11-28,3.204,3.069,2.843,2.827,3.088


In [3]:
# Helper functions
def moving_average(series, window):
    return series.rolling(window=window).mean()

def exponential_moving_average(series, window):
    return series.ewm(span=window, adjust=False).mean()

def double_exponential_moving_average(series, window):
    ema1 = exponential_moving_average(series, window)
    ema2 = exponential_moving_average(ema1, window)
    return 2 * ema1 - ema2

def calculate_rsi(series, window=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def calculate_roc(series, window=14):
    return ((series - series.shift(window)) / series.shift(window)) * 100

def calculate_keltner_channel(series, window=20, atr_multiplier=2):
    ema = exponential_moving_average(series, window)
    atr = series.diff().abs().rolling(window).mean()  # Simplified ATR
    upper = ema + atr_multiplier * atr
    lower = ema - atr_multiplier * atr
    return upper, lower

def calculate_bollinger_bands(series, window=20, multiplier=2):
    sma = moving_average(series, window)
    std = series.rolling(window=window).std()
    upper_band = sma + multiplier * std
    lower_band = sma - multiplier * std
    return upper_band, lower_band

def calculate_macd(series, fast=12, slow=26, signal=9):
    ema_fast = exponential_moving_average(series, fast)
    ema_slow = exponential_moving_average(series, slow)
    macd = ema_fast - ema_slow
    signal_line = exponential_moving_average(macd, signal)
    return macd, signal_line

def calculate_bias(series, window=20):
    sma = moving_average(series, window)
    bias = (series - sma) / sma * 100
    return bias

def calculate_z_score(series, window=20):
    mean = series.rolling(window=window).mean()
    std = series.rolling(window=window).std()
    return (series - mean) / std

# Function to calculate technical indicators
def calculate_technical_indicators(data, price_column):
    indicators = pd.DataFrame()
    indicators['Dates'] = data['Dates']
    
    # Trend Indicators
    indicators['SMA_50'] = moving_average(data[price_column], window=50)
    indicators['SMA_200'] = moving_average(data[price_column], window=200)
    indicators['SMA_crossover'] = np.where(indicators['SMA_50'] > indicators['SMA_200'], 1, 0)
    indicators['DEMA'] = double_exponential_moving_average(data[price_column], window=20)
    indicators['MACD'], indicators['MACD_Signal'] = calculate_macd(data[price_column])
    
    # Oscillators
    indicators['RSI'] = calculate_rsi(data[price_column], window=14)
    indicators['ROC'] = calculate_roc(data[price_column], window=14)
    indicators['BIAS'] = calculate_bias(data[price_column], window=20)
    indicators['Z_Score'] = calculate_z_score(data[price_column], window=20)
    
    # Channel Indicators
    indicators['Keltner_upper'], indicators['Keltner_lower'] = calculate_keltner_channel(data[price_column], window=20)
    indicators['Bollinger_upper'], indicators['Bollinger_lower'] = calculate_bollinger_bands(data[price_column], window=20)
    
    return indicators

In [4]:
# Apply the function for each commodity and save to separate CSV files
commodity_columns = ['NG1 Comdty', 'NG2 Comdty', 'NG3 Comdty', 'NG4 Comdty', 'NG6 Comdty']
for column in commodity_columns:
    indicators_df = calculate_technical_indicators(data, column)
    file_name = f"./data/{column.replace(' ', '_')}_technical_indicators.csv"
    indicators_df.to_csv(file_name, index=False)